# Auswertung Adliswil

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
path = "All_Datas/daten_adliswil.csv"
df = pd.read_csv(path)

In [4]:
path2 = "All_Datas/Parlamentarier_Adliswil.csv"
parlamentarier_df = pd.read_csv(path2)

In [6]:
parlamentarier_df

,Name,Vorname,Partei
0,Abstreiter-Knecht,Brigitte,CVP
1,Acar,Sait,SP
2,Aebli,Daniel,SVP
3,Bach,Vera,FW
4,Baldegger,Harry,FW
5,Barco Greiner,Gabriella,SP
6,Barmettler,Peter,FDP
7,Biagioli,Ann-Kathrin,GLP
8,Blümli,Andrea,CVP
9,Broggini,Angela,GRÜNE


## 1. Welche Parteien reichen die häufigsten Vorstösse ein?